In [3]:
import pandas as pd
import numpy as np
import pymysql
import pymongo
import random
import nltk
import re

In [4]:
import pandas as pd
import pymysql
import pymongo
import random
import nltk
import re

nltk.download('punkt')

# Setup connections
mysql_conn = pymysql.connect(
    host='localhost',
    user='root',
    password='password',
    database='chatdb'
)
mysql_cursor = mysql_conn.cursor()
mongo_client = pymongo.MongoClient('mongodb://localhost:27017/')
mongo_db = mongo_client['chatdb']

# Query templates
query_templates = [
    "SELECT {col1}, SUM({col2}) FROM {table} GROUP BY {col1}",
    "SELECT {col1}, COUNT(*) FROM {table} WHERE {col2} > {value}",
    "SELECT {col1}, AVG({col2}) FROM {table} ORDER BY {col2} DESC LIMIT {limit}"
]

# Function implementations
def upload_csv_to_databases(csv_path, table_name):
    """Uploads a CSV file to MySQL and MongoDB."""
    df = pd.read_csv(csv_path)
    
    # MySQL upload
    create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        {', '.join([f'{col} VARCHAR(255)' for col in df.columns])}
    )
    """
    mysql_cursor.execute(create_table_query)
    mysql_conn.commit()

    for _, row in df.iterrows():
        insert_query = f"""
        INSERT INTO {table_name} VALUES ({', '.join(['%s' for _ in row])})
        """
        mysql_cursor.execute(insert_query, tuple(row))
    mysql_conn.commit()

    # MongoDB upload
    mongo_db[table_name].insert_many(df.to_dict('records'))
    print(f"Data from {csv_path} uploaded to MySQL and MongoDB.")

def explore_mysql_database():
    """Displays MySQL tables and attributes."""
    mysql_cursor.execute("SHOW TABLES")
    tables = mysql_cursor.fetchall()
    for table in tables:
        print(f"Table: {table[0]}")
        mysql_cursor.execute(f"DESCRIBE {table[0]}")
        print("Columns:", mysql_cursor.fetchall())
        mysql_cursor.execute(f"SELECT * FROM {table[0]} LIMIT 5")
        print("Sample Data:", mysql_cursor.fetchall())

def explore_mongodb_database():
    """Displays MongoDB collections and attributes."""
    collections = mongo_db.list_collection_names()
    for collection in collections:
        print(f"Collection: {collection}")
        print("Sample Data:", list(mongo_db[collection].find().limit(5)))

def generate_sample_queries(table, columns):
    """Generates sample SQL queries dynamically."""
    samples = []
    for template in query_templates:
        samples.append(template.format(
            col1=random.choice(columns),
            col2=random.choice(columns),
            table=table,
            value=random.randint(1, 100),
            limit=random.randint(5, 10)
        ))
    return samples

def parse_natural_language_query(query):
    """Parses natural language queries into patterns."""
    tokens = nltk.word_tokenize(query.lower())
    patterns = {
        "total <A> by <B>": r"total (\w+) by (\w+)",
        "find <A> where <B>": r"find (\w+) where (\w+)"
    }
    for description, pattern in patterns.items():
        match = re.search(pattern, query)
        if match:
            return description, match.groups()
    return "No matching pattern", None

def execute_mysql_query(query):
    """Executes a MySQL query and returns results."""
    mysql_cursor.execute(query)
    return mysql_cursor.fetchall()

def execute_mongodb_query(collection_name, query_filter):
    """Executes a MongoDB query and returns results."""
    return list(mongo_db[collection_name].find(query_filter))

# Main terminal-based interaction
def main():
    print("Welcome to ChatDB! Your interactive database assistant.")
    while True:
        print("\nOptions:")
        print("1. Upload a CSV file to databases")
        print("2. Explore MySQL database")
        print("3. Explore MongoDB database")
        print("4. Generate sample queries")
        print("5. Execute a natural language query")
        print("6. Exit")
        choice = input("Enter your choice: ")

        if choice == '1':
            csv_path = input("Enter the path to the CSV file: ")
            upload_csv_to_databases(csv_path)
        elif choice == '2':
            print("Exploring MySQL database...")
            explore_mysql_database()
        elif choice == '3':
            print("Exploring MongoDB database...")
            explore_mongodb_database()
        elif choice == '4':
            table_name = input("Enter the table name for sample queries: ")
            columns = input("Enter the columns in the table (comma-separated): ").split(',')
            print("Generated Queries:")
            for query in generate_sample_queries(table_name, columns):
                print(query)
        elif choice == '5':
            user_query = input("Enter your natural language query: ")
            description, params = parse_natural_language_query(user_query)
            if description == "No matching pattern":
                print("Sorry, I couldn't interpret your query.")
            else:
                print(f"Interpreted Query: {description}")
                print(f"Parameters: {params}")
        elif choice == '6':
            print("Exiting ChatDB. Goodbye!")
            break
        else:
            print("Invalid choice. Please try again.")

# Run the CLI
if __name__ == '__main__':
    main()


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/dongchenxi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


OperationalError: (2003, "Can't connect to MySQL server on 'localhost' ([Errno 61] Connection refused)")

In [10]:
import pandas as pd
import pymysql
import pymongo

# MySQL connection setup
mysql_conn = pymysql.connect(
    host='localhost',
    user='root',
    password='dcx20021110@'
)
mysql_cursor = mysql_conn.cursor()

database_name = "chatdb"
create_database_query = f"CREATE DATABASE IF NOT EXISTS {database_name}"
mysql_cursor.execute(create_database_query)
print(f"Database '{database_name}' created or already exists.")

# MongoDB connection setup
mongo_client = pymongo.MongoClient('mongodb://localhost:27017/')
mongo_db = mongo_client['chatdb']

def infer_mysql_data_type(series):
    """
    Infer MySQL data type from a pandas Series.
    """
    if pd.api.types.is_integer_dtype(series):
        return "INT"
    elif pd.api.types.is_float_dtype(series):
        return "FLOAT"
    elif pd.api.types.is_bool_dtype(series):
        return "BOOLEAN"
    elif pd.api.types.is_datetime64_any_dtype(series):
        return "DATETIME"
    else:
        # Default to VARCHAR with a length estimate
        max_length = series.dropna().astype(str).map(len).max()
        return f"VARCHAR({max(255, max_length)})"

def upload_csv_to_databases(csv_path, table_name):
    """
    Uploads a CSV file to MySQL and MongoDB, using inferred data types for MySQL.
    """
    # Load CSV into pandas DataFrame
    df = pd.read_csv(csv_path)

    # Infer column data types for MySQL
    column_definitions = []
    for column in df.columns:
        data_type = infer_mysql_data_type(df[column])
        column_definitions.append(f"{column} {data_type}")

    

    """
    print column_definitions to users,
    prompt user to input first table name and columns within table
    create table and insert with those columns

    repeat
    
    """

    


    # Create table in MySQL with inferred data types
    create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        {', '.join(column_definitions)}
    )
    """
    mysql_cursor.execute(create_table_query)
    mysql_conn.commit()

    # Insert rows into MySQL
    for _, row in df.iterrows():
        insert_query = f"""
        INSERT INTO {table_name} VALUES ({', '.join(['%s' for _ in row])})
        """
        mysql_cursor.execute(insert_query, tuple(row))
    mysql_conn.commit()

    # Upload to MongoDB
    mongo_db[table_name].insert_many(df.to_dict('records'))
    print(f"Data from {csv_path} uploaded to MySQL and MongoDB with inferred data types.")

# Example usage
# upload_csv_to_databases('example.csv', 'example_table')


Database 'chatdb' created or already exists.


In [ ]:
upload_csv_to_databases()

In [13]:
import pandas as pd
import pymysql
import pymongo

# MySQL connection setup
mysql_conn = pymysql.connect(
    host='localhost',
    user='root',
    password='dcx20021110@',
    database='chatdb'
)
mysql_cursor = mysql_conn.cursor()

# MongoDB connection setup
mongo_client = pymongo.MongoClient('mongodb://localhost:27017/')
mongo_db = mongo_client['chatdb']

def create_database_if_not_exists(database_name):
    """
    Creates a database if it doesn't exist.
    """
    mysql_cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database_name}")
    mysql_cursor.execute(f"USE {database_name}")
    mysql_conn.commit()

def infer_mysql_data_type(series):
    """
    Infer MySQL data type from a pandas Series.
    """
    if pd.api.types.is_integer_dtype(series):
        return "INT"
    elif pd.api.types.is_float_dtype(series):
        return "FLOAT"
    elif pd.api.types.is_bool_dtype(series):
        return "BOOLEAN"
    elif pd.api.types.is_datetime64_any_dtype(series):
        return "DATETIME"
    else:
        # Default to VARCHAR with a length estimate
        max_length = series.dropna().astype(str).map(len).max()
        return f"VARCHAR({max(255, max_length)})"

def upload_csv_to_databases(csv_path):
    """
    Uploads a CSV file to MySQL and MongoDB, using user-defined table names and column selections.
    """
    # Ensure the database exists
    create_database_if_not_exists('chatdb')

    # Load CSV into pandas DataFrame
    df = pd.read_csv(csv_path)

    # Infer column data types for MySQL
    column_definitions = []
    for column in df.columns:
        data_type = infer_mysql_data_type(df[column])
        column_definitions.append(f"{column} {data_type}")

    # Loop to allow user input for table creation and column selection
    while True:
        print("\nColumn Definitions:")
        for i, col_def in enumerate(column_definitions):
            print(f"{i + 1}. {col_def}")

        table_name = input("\nEnter a table name (or type 'exit' to finish): ")
        if table_name.lower() == 'exit':
            break

        selected_columns = input("Enter the column numbers to include in the table, separated by commas (e.g., 1,2,3): ")
        selected_columns = [int(i.strip()) - 1 for i in selected_columns.split(',')]

        # Create table with user-selected columns
        selected_definitions = [column_definitions[i] for i in selected_columns]
        create_table_query = f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            {', '.join(selected_definitions)}
        )
        """
        mysql_cursor.execute(create_table_query)
        mysql_conn.commit()
        print(f"Table '{table_name}' created successfully.")

        # Insert rows into the selected columns
        selected_column_names = [df.columns[i] for i in selected_columns]
        for _, row in df[selected_column_names].iterrows():
            insert_query = f"""
            INSERT INTO {table_name} VALUES ({', '.join(['%s' for _ in selected_columns])})
            """
            mysql_cursor.execute(insert_query, tuple(row))
        mysql_conn.commit()
        print(f"Data inserted into table '{table_name}'.")

        # Upload to MongoDB
        mongo_db[table_name].insert_many(df[selected_column_names].to_dict('records'))
        print(f"Data from selected columns uploaded to MongoDB collection '{table_name}'.")

    print("All data uploaded. Exiting program.")

# Example usage
# upload_csv_to_databases('example.csv')

In [16]:
upload_csv_to_databases('vgsales.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'vgsales.csv'

In [17]:
import os
print(os.getcwd())

/
